In [1]:
import torch,torchvision
from torch import nn,optim
from torch.optim import Adam 

In [2]:
class MNIST(nn.Module):
    
    def __init__(self):
        super(MNIST,self).__init__()
        self.model=nn.Sequential(
        nn.Conv2d(1,6,5,padding=2),     #input,output,filter dimensions the padding and stride
        nn.ReLU(),
        nn.AvgPool2d(2,stride=2),
        
        nn.Conv2d(6,16,5,padding=0),
        nn.ReLU(),
        nn.AvgPool2d(2,stride=2),
        
        nn.Flatten(),
        nn.Linear(400,120),           #16*5*5
        nn.Linear(120,84),
        nn.Linear(84,10)   #total no.of digits=10
        )
    
    
    def forward(self,out):
        x=self.model(out)
        return x
    
model = MNIST()
model.load_state_dict(torch.load('model1.pt'))
model.eval()

MNIST(
  (model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): Linear(in_features=120, out_features=84, bias=True)
    (9): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [3]:
import cv2
import numpy as np
import torch,torchvision
from torchvision import transforms


load_from_sys = True

if load_from_sys:
	hsv_value = np.load('hsv_value1.npy')       #value=array([[10,50,70], [ 100, 255, 255]], dtype=int64)

cap = cv2.VideoCapture(0)
cap.set(3,1280)                            #width,resolution
cap.set(4,720)

def prediction(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)  #changing to gray
    image=cv2.resize(image,(28,28))             #resizing to 28,28
    image=image.astype(np.float32)/255.         #changing to float 32(to send to model)
    img = torch.tensor(image).unsqueeze(0).unsqueeze(0).to('cpu')  

    
    with torch.no_grad():
        pred=model(img)
    return np.argmax(pred)    
        

kernel = np.ones((5,5), np.uint8)            #filter to smoothen the image

canvas = None

x1 = 0
y1 = 0

noise_thresh = 800      
while True:
	ret, frame = cap.read()     #boolean value,frame read
    
	if not ret:
		print('camera cant be opened')
		break
    
	frame = cv2.flip(frame, 1)            # 1 means flipping arond y axis

	if canvas is None:
		canvas = np.zeros_like(frame)         #creating black screen of same size
    
    
	hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)       #converting to hsv

	if load_from_sys:
		lower_range = hsv_value[0]               #70,0,59
		upper_range = hsv_value[1]               #179,255,255
     
	mask = cv2.inRange(frame,lower_range, upper_range)     #mask of neon orange

	mask = cv2.erode(mask, kernel, iterations = 1)
	mask = cv2.dilate(mask, kernel, iterations = 2)          #erosion and dilation(processing of mask)

    
	contours, heirarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    
	if contours  and cv2.contourArea(max(contours, key = cv2.contourArea)) > noise_thresh:
		c = max(contours, key = cv2.contourArea)
		x2, y2 ,w, h = cv2.boundingRect(c)

		if x1 == 0 and y1 == 0:
			x1,y1 = x2,y2

		else:
			canvas = cv2.drawContours(frame,contours,-1,(0,255,0),3) 
            


	else:
        
		x1,y1 = 0, 0
        
    
	frame = cv2.add(frame, canvas)
    
	stacked = np.hstack((canvas, frame))                 #to display the canvas and frame together

	cv2.imshow('Screen_Pen', cv2.resize(stacked, None, fx = 0.6, fy = 0.6))     #minimizing th imagw

	if cv2.waitKey(1) == 13:
		break

	#Clear the canvas when 'c' is pressed
    
	if cv2.waitKey(1) & 0xFF == ord('c'):           
		canvas = None

	if cv2.waitKey(1) & 0xFF == ord('o'):
		a=prediction(canvas,model)
		cv2.putText(canvas, "Convolution Neural Network:  " + str(a), (20, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
					255,3)  
        
        #image,text,coordinates,dtyle,thickness,colour,linetype
        

        
cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:650: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'
